<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/load_all_channel_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [0]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [0]:
!pip install -q 'canopy==5.3'

In [0]:
import canopy
import logging

logging.basicConfig(level=logging.INFO)

### Authenticate

In [0]:
authentication_data = canopy.prompt_for_authentication()

def create_session():
    return canopy.Session(authentication_data)

# Set Up Example

### Run a simulation using default configs

In [32]:
default_car_name = 'Canopy F1 Car 2019'
default_weather_name = '25 deg, dry'

async with create_session() as session:
   
    logging.info('Loading configs...')
    
    default_car = await canopy.load_default_config(session, 'car', default_car_name)
    default_weather = await canopy.load_default_config(session, 'weather', default_weather_name)

    logging.info('Creating study...')
    
    study_id = await canopy.create_study(
        session,
        'apexSim',
        'Python API Test Study',
        [
            default_car,
            default_weather,
        ])

    logging.info('Waiting for study to complete...')

    wait_result = await canopy.wait_for_study(
        session,
        study_id,
        timeout_seconds=60)

logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Job Count: {}'.format(wait_result.data.job_count))
logging.info('Succeded Job Count: {}'.format(wait_result.data.succeeded_job_count))

INFO:root:Loading configs...
INFO:root:Creating study...
INFO:root:Waiting for study to complete...
INFO:root:Sim Version: 1.3020
INFO:root:Job Count: 1
INFO:root:Succeded Job Count: 1


# Example: Load all channel data
_Note:_ For performance reasons it is usually recommended to load only the channels you need.

In [45]:
# First get the vector metadata, which contains all the channel information.
async with create_session() as session:
    job_metadata = await canopy.load_study_job(session, study_id, 'ApexSim', include_vector_metadata=True)
job_metadata.vector_metadata

INFO:canopy.load_study_job:Loading job index 0


,units,description,NPtsInChannel,xDomainName
name,,,,
xCar,m,x position of the car centre of gravity in the...,42,vCar
yCar,m,y position of the car centre of gravity in the...,42,vCar
zCar,m,z position of the car centre of gravity in the...,42,vCar
aRoll,rad,Roll angle of the car in the world coordinate ...,42,vCar
aPitch,rad,Pitch angle of the car in the world coordinate...,42,vCar
...,...,...,...,...
aSlipTyreR,rad,Average slip angle of the rear tyres.,42,vCar
zTrackFL,m,Track height at the front left contact patch.,42,vCar
zTrackFR,m,Track height at the front right contact patch.,42,vCar


In [34]:
# Next extract the list of channels. 
# The vector_metadata is a DataFrame indexed by the channel name.
all_channels = job_metadata.vector_metadata.index
all_channels

Index(['xCar', 'yCar', 'zCar', 'aRoll', 'aPitch', 'aYaw', 'vCar', 'dxCar',
       'dyCar', 'dzCar',
       ...
       'nWheelRR', 'dnWheelRR', 'aSteerKinematicRR', 'aSteerTotalRR',
       'aSlipTyreF', 'aSlipTyreR', 'zTrackFL', 'zTrackFR', 'zTrackRL',
       'zTrackRR'],
      dtype='object', name='name', length=382)

In [35]:
# Finally load the entire list of channels.
async with create_session() as session:
    job_data = await canopy.load_study_job(session, study_id, 'ApexSim', channel_names=all_channels)

logging.info('Loaded {} channels'.format(len(job_data.vector_data.columns)))

INFO:canopy.load_study_job:Loading job index 0
INFO:root:Loaded 382 channels


In [36]:
# The DataFrame containing all the channel data.
job_data.vector_data

,xCar,yCar,zCar,aRoll,aPitch,aYaw,vCar,dxCar,dyCar,dzCar,nRoll,nPitch,nYaw,gLong,gLat,gVert,dnRoll,dnPitch,dnYaw,aSlipCar,aSteerWheel,aSteerL,aSteerR,gLongAccelerometer,gLatAccelerometer,gVertAccelerometer,xRackL,xRackR,d2xRackL,d2xRackR,FRackL,FRackR,hRideF,hRideR,aInGGPlane,rLatBalF,cUnderOverSteer,PEnergyKinetic,hRollCentreF,hRollCentreR,...,dFyTyre_daSlipRL,MyRollingResistanceRL,nWheelRL,dnWheelRL,aSteerKinematicRL,aSteerTotalRL,aSlipTyreRR,rSlipTyreRR,aCamberTyreRR,rTyreLoadedRR,xTyreSquashRR,rTyreRollingRadiusRR,xBumpRR,kHeaveTyreRR,PTyreWearRR,PTyreWearLatRR,PTyreWearLongRR,rGripTyreWearRR,rGripTyreRR,rGripTemperatureRR,FxTyreRR,FyTyreRR,FzTyreRR,MxTyreRR,MyTyreRR,MzTyreRR,FxyTyreRR,dFxTyre_drSlipRR,dFyTyre_daSlipRR,MyRollingResistanceRR,nWheelRR,dnWheelRR,aSteerKinematicRR,aSteerTotalRR,aSlipTyreF,aSlipTyreR,zTrackFL,zTrackFR,zTrackRL,zTrackRR
0,0.0,0.0,-0.329211,-0.013374,-0.028220,0.0,9.894445,9.898387,1.421950,0.0,0.0,0.0,1.115020,-3.336827,10.782156,0.238413,0.016614,0.035070,4.209975e-08,0.143092,4.861907,0.419794,0.405675,-3.613632,10.913301,-9.566804,-0.038134,0.038538,1.777039e-09,4.048749e-10,-5557.940430,1520.595825,0.023579,0.125159,1.870925,0.330607,0.003171,-2.455206e+04,0.051551,0.031668,...,62577.109375,1.373589,31.928099,-5.854145,0.003029,0.003029,-0.039399,-0.003411,-0.023205,0.331409,0.003591,0.335022,0.053975,237007.468750,297.920593,295.625183,2.295432,1.0,1.0,1.0,-74.643211,832.300110,-878.514282,-154.230743,0.0,-43.399548,835.640503,23142.117188,17499.847656,0.324083,26.909157,-14.074287,-0.003288,-0.003288,-0.066443,-0.039141,0.0,0.0,0.0,0.0
1,0.0,0.0,-0.328920,-0.014789,-0.028292,0.0,11.929656,11.934432,1.252731,0.0,0.0,0.0,1.170616,-3.914652,13.709257,0.313561,0.020250,0.038753,-7.500622e-09,0.105029,4.685953,0.401159,0.391341,-4.192159,13.854278,-9.491441,-0.036555,0.037095,-5.491771e-07,1.294986e-07,-7349.648438,1949.438843,0.023153,0.124990,1.848942,0.408030,0.012541,-3.473244e+04,0.053720,0.032958,...,50524.828125,1.719216,38.090752,-6.580432,0.003019,0.003019,-0.055035,-0.003866,-0.024566,0.331823,0.003177,0.335032,0.054481,236256.968750,581.617371,578.584351,3.033049,1.0,1.0,1.0,-71.344398,955.932983,-783.010559,-145.370651,0.0,-45.135586,958.591614,20561.753906,12035.944336,0.351166,32.825783,-16.798317,-0.003302,-0.003302,-0.111064,-0.054099,0.0,0.0,0.0,0.0
2,0.0,0.0,-0.328392,-0.013976,-0.028384,0.0,13.964086,13.969712,0.920404,0.0,0.0,0.0,1.074219,-3.809538,14.816443,0.315254,0.016114,0.032739,1.577957e-08,0.066206,3.787467,0.317803,0.320089,-4.087945,14.953486,-9.489837,-0.029319,0.029987,-2.727417e-07,3.031244e-08,-8206.023438,1530.826782,0.022441,0.124609,1.822461,0.473971,0.011882,-3.956650e+04,0.054756,0.032080,...,44932.542969,1.971326,43.853710,-6.769620,0.003023,0.003023,-0.058983,-0.008945,-0.023809,0.331356,0.003644,0.335045,0.053940,237124.343750,902.741943,881.070740,21.671206,1.0,1.0,1.0,-185.895172,1146.195679,-896.666321,-157.728149,0.0,-76.820702,1161.172485,23334.111328,12729.846680,0.478400,38.897717,-15.980435,-0.003288,-0.003288,-0.108912,-0.058662,0.0,0.0,0.0,0.0
3,0.0,0.0,-0.327895,-0.013501,-0.028429,0.0,15.981847,15.988308,0.611569,0.0,0.0,0.0,0.984572,-3.803476,15.614969,0.319006,0.013077,0.027546,3.393830e-08,0.038628,3.178591,0.261705,0.271100,-4.082326,15.747361,-9.486136,-0.024394,0.025163,4.112396e-10,1.084461e-11,-9005.466797,1311.044189,0.021854,0.124184,1.809722,0.513248,0.012477,-4.521374e+04,0.055273,0.031511,...,41837.238281,2.240722,49.577271,-7.116454,0.003023,0.003023,-0.061111,-0.012619,-0.023378,0.330984,0.004016,0.335060,0.053521,237622.546875,1240.809448,1186.552856,54.256615,1.0,1.0,1.0,-285.761932,1290.499390,-988.421875,-165.782318,0.0,-102.914757,1321.759644,25403.433594,13401.417969,0.610146,44.904331,-15.596311,-0.003278,-0.003278,-0.111661,-0.061352,0.0,0.0,0.0,0.0
4,0.0,0.0,-0.327382,-0.013282,-0.028454,0.0,17.989611,17.996895,0.334292,0.0,0.0,0.0,0.910606,-3.881668,16.317331,0.327231,0.011004,0.023584,2.270305e-08,0.018957,2.748406,

In [37]:
# Request a channel in specific units.
job_data.vector_as('hRideF', 'mm')

0     23.579136
1     23.152531
2     22.441225
3     21.853617
4     21.290359
5     20.717617
6     20.139326
7     19.573612
8     19.165575
9     18.524010
10    17.838722
11    17.121336
12    16.375395
13    15.612660
14    14.844961
15    14.148068
16    13.394130
17    12.591953
18    11.773566
19    10.901817
20    10.006719
21     9.095339
22     8.211351
23     7.243102
24     6.249730
25     5.232016
26     4.191138
27     3.163227
28     2.091182
29     0.983876
30    -0.147225
31    -1.304280
32    -2.462396
33    -3.671250
34    -4.910093
35    -6.178942
36    -7.453597
37    -8.757462
38    -9.866764
39   -10.362043
40   -10.723705
41   -11.074367
Name: hRideF, dtype: float32